In [43]:
import pandas as pd
pd.set_option('display.float_format', '{:.5f}'.format)
import numpy as np

In [44]:
ticker="soxs"
df=pd.read_csv(f"data/raw/{ticker}_20250103_20250131.csv",engine="pyarrow", sep = ',')
df["Date-Time"] = df["Date-Time"].dt.tz_convert("America/New_York")
df.head(3)

,#RIC,Domain,Date-Time,Type,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,...,L23-AskPrice,L23-AskSize,L24-BidPrice,L24-BidSize,L24-AskPrice,L24-AskSize,L25-BidPrice,L25-BidSize,L25-AskPrice,L25-AskSize
0,SOXS.K,Market Price,2025-01-03 00:10:00.511401066-05:00,Normalized LL2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SOXS.K,Market Price,2025-01-03 07:00:00.026209690-05:00,Normalized LL2,21.71000,100.00000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SOXS.K,Market Price,2025-01-03 07:00:00.026209690-05:00,Normalized LL2,21.71000,100.00000,NaN,NaN,21.70000,100.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
pd.DataFrame(pd.to_datetime(df["Date-Time"]).dt.date.unique())

,0
0,2025-01-03
1,2025-01-06
2,2025-01-07
3,2025-01-08
4,2025-01-10
5,2025-01-13
6,2025-01-14
7,2025-01-15
8,2025-01-16
9,2025-01-17


In [46]:
df["Date"] = pd.to_datetime(df["Date-Time"]).dt.date
grouped = df.groupby("Date").size().reset_index(name="Count")


In [47]:
# Filter only regular market hours (09:30–16:00 Eastern)
df_market = df[
    (df["Date-Time"].dt.time >= pd.to_datetime("09:40:00").time()) &
    (df["Date-Time"].dt.time <= pd.to_datetime("15:50:00").time())
]

In [48]:
base_cols = ["#RIC", "Date-Time"]
#base_cols = ["#RIC"]
# Dynamically build list for L1–L10
levels = []
for i in range(1, 11):  # L1–L10 inclusive
    levels += [
        f"L{i}-BidPrice", f"L{i}-BidSize",
        f"L{i}-AskPrice", f"L{i}-AskSize"
    ]


In [49]:
df_market_l10 = df_market[base_cols + levels]

In [50]:
df_market_l10.head(10)

,#RIC,Date-Time,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,L2-AskSize,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
36747,SOXS.K,2025-01-03 09:40:00.006514971-05:00,21.30000,100.00000,21.31000,2600.00000,21.29000,800.00000,21.32000,3200.00000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36748,SOXS.K,2025-01-03 09:40:00.006514971-05:00,21.29000,800.00000,21.31000,2600.00000,21.28000,900.00000,21.32000,3200.00000,...,21.38000,1900.00000,21.21000,100.00000,21.43000,100.00000,21.20000,100.00000,21.44000,2300.00000
36749,SOXS.K,2025-01-03 09:40:00.016467552-05:00,21.30000,600.00000,21.31000,2600.00000,21.29000,800.00000,21.32000,3200.00000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36750,SOXS.K,2025-01-03 09:40:00.016467552-05:00,21.30000,700.00000,21.31000,2600.00000,21.29000,800.00000,21.32000,3200.00000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36751,SOXS.K,2025-01-03 09:40:00.021517897-05:00,21.30000,700.00000,21.31000,2600.00000,21.29000,800.00000,21.32000,3200.00000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36752,SOXS.K,2025-01-03 09:40:00.056315268-05:00,21.30000,700.00000,21.31000,2000.00000,21.29000,800.00000,21.32000,3200.00000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36753,SOXS.K,2025-01-03 09:40:00.069284280-05:00,21.30000,1000.00000,21.31000,2000.00000,21.29000,800.00000,21.32000,3200.00000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36754,SOXS.K,2025-01-03 09:40:00.069284280-05:00,21.30000,1000.00000,21.31000,1900.00000,21.29000,800.00000,21.32000,3200.00000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36755,SOXS.K,2025-01-03 09:40:00.076256984-05:00,21.30000,1000.00000,21.31000,1900.00000,21.29000,800.00000,21.32000,1300.00000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36756,SOXS.K,2025-01-03 09:40:00.076256984-05:00,21.30000,1100.00000,21.31000,1900.00000,21.29000,800.00000,21.32000,1300.00000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000


In [51]:
price_cols = df_market_l10.filter(regex=r'^L(?:10|[1-9])-(?:Bid|Ask)Price$').columns

# Keep only rows where ANY price column changed vs the previous row
changed = df_market_l10[price_cols].ne(df_market_l10[price_cols].shift()).any(axis=1)
df_filtered = df_market_l10[changed].copy()

In [52]:
print(df_filtered.size)
print(df_market_l10.size)

48842178
587923518


In [53]:
df_market_l10=df_filtered

In [54]:
df_market_l10["ms_timestamp"] = df_market_l10["Date-Time"].dt.floor("ns")
df_market_l10 = (
   df_market_l10.drop_duplicates(subset=["#RIC", "ms_timestamp"], keep="last")
     .drop(columns="ms_timestamp")
)
df_market_l10=df_market_l10.dropna()

In [55]:
df_market_l10.size

40353810

In [56]:
df_market_l10.head(10)

,#RIC,Date-Time,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,L2-AskSize,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
36748,SOXS.K,2025-01-03 09:40:00.006514971-05:00,21.29000,800.00000,21.31000,2600.00000,21.28000,900.00000,21.32000,3200.00000,...,21.38000,1900.00000,21.21000,100.00000,21.43000,100.00000,21.20000,100.00000,21.44000,2300.00000
36749,SOXS.K,2025-01-03 09:40:00.016467552-05:00,21.30000,600.00000,21.31000,2600.00000,21.29000,800.00000,21.32000,3200.00000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36758,SOXS.K,2025-01-03 09:40:00.076256984-05:00,21.30000,1100.00000,21.32000,1300.00000,21.29000,800.00000,21.33000,3200.00000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36762,SOXS.K,2025-01-03 09:40:00.081235935-05:00,21.31000,600.00000,21.32000,1300.00000,21.30000,2400.00000,21.33000,3200.00000,...,21.43000,100.00000,21.23000,2000.00000,21.44000,2300.00000,21.22000,100.00000,21.52000,2300.00000
36777,SOXS.K,2025-01-03 09:40:00.116134193-05:00,21.30000,800.00000,21.32000,3900.00000,21.29000,2700.00000,21.33000,3200.00000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36784,SOXS.K,2025-01-03 09:40:00.126062523-05:00,21.31000,600.00000,21.32000,3900.00000,21.30000,700.00000,21.33000,3200.00000,...,21.43000,100.00000,21.23000,2000.00000,21.44000,2300.00000,21.22000,100.00000,21.52000,2300.00000
36789,SOXS.K,2025-01-03 09:40:00.131044286-05:00,21.30000,700.00000,21.32000,3900.00000,21.29000,800.00000,21.33000,3200.00000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36791,SOXS.K,2025-01-03 09:40:00.156879201-05:00,21.30000,1900.00000,21.31000,1200.00000,21.29000,800.00000,21.32000,3900.00000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36814,SOXS.K,2025-01-03 09:40:00.241564103-05:00,21.30000,1000.00000,21.32000,3200.00000,21.29000,800.00000,21.33000,3200.00000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36821,SOXS.K,2025-01-03 09:40:00.346022423-05:00,21.30000,1000.00000,21.31000,600.00000,21.29000,800.00000,21.32000,3200.00000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000


In [57]:
#df_market_l10.drop("mid_price",axis=1, inplace=True)
#df_market_l10.drop("abs.spread",axis=1, inplace=True)
#df_market_l10.drop("rel.spread",axis=1, inplace=True)

In [58]:
df_market_l10.insert(
            2, "mid_price", (df_market_l10["L1-BidPrice"] + df_market_l10["L1-AskPrice"]) / 2
        )  # Add the mid-price column to the orderbook dataframe.
df_market_l10.insert(
            2, "abs.spread", (df_market_l10["L1-AskPrice"] - df_market_l10["L1-BidPrice"])
        )
df_market_l10.insert(
            2, "rel.spread", (df_market_l10["L1-AskPrice"] - df_market_l10["L1-BidPrice"])/df_market_l10["mid_price"]
        )

In [59]:
df_market_l10.head(10)

,#RIC,Date-Time,rel.spread,abs.spread,mid_price,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
36748,SOXS.K,2025-01-03 09:40:00.006514971-05:00,0.00094,0.02000,21.30000,21.29000,800.00000,21.31000,2600.00000,21.28000,...,21.38000,1900.00000,21.21000,100.00000,21.43000,100.00000,21.20000,100.00000,21.44000,2300.00000
36749,SOXS.K,2025-01-03 09:40:00.016467552-05:00,0.00047,0.01000,21.30500,21.30000,600.00000,21.31000,2600.00000,21.29000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36758,SOXS.K,2025-01-03 09:40:00.076256984-05:00,0.00094,0.02000,21.31000,21.30000,1100.00000,21.32000,1300.00000,21.29000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36762,SOXS.K,2025-01-03 09:40:00.081235935-05:00,0.00047,0.01000,21.31500,21.31000,600.00000,21.32000,1300.00000,21.30000,...,21.43000,100.00000,21.23000,2000.00000,21.44000,2300.00000,21.22000,100.00000,21.52000,2300.00000
36777,SOXS.K,2025-01-03 09:40:00.116134193-05:00,0.00094,0.02000,21.31000,21.30000,800.00000,21.32000,3900.00000,21.29000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36784,SOXS.K,2025-01-03 09:40:00.126062523-05:00,0.00047,0.01000,21.31500,21.31000,600.00000,21.32000,3900.00000,21.30000,...,21.43000,100.00000,21.23000,2000.00000,21.44000,2300.00000,21.22000,100.00000,21.52000,2300.00000
36789,SOXS.K,2025-01-03 09:40:00.131044286-05:00,0.00094,0.02000,21.31000,21.30000,700.00000,21.32000,3900.00000,21.29000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36791,SOXS.K,2025-01-03 09:40:00.156879201-05:00,0.00047,0.01000,21.30500,21.30000,1900.00000,21.31000,1200.00000,21.29000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36814,SOXS.K,2025-01-03 09:40:00.241564103-05:00,0.00094,0.02000,21.31000,21.30000,1000.00000,21.32000,3200.00000,21.29000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36821,SOXS.K,2025-01-03 09:40:00.346022423-05:00,0.00047,0.01000,21.30500,21.30000,1000.00000,21.31000,600.00000,21.29000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000


In [60]:
print(sum(round(df_market_l10["abs.spread"],2)==float(0.01)))
tick_size=sum(round(df_market_l10["abs.spread"],2)==float(0.01))/df_market_l10.size
print(tick_size)

516409
0.011943896582090597


In [61]:
df_market_l10.describe()

,rel.spread,abs.spread,mid_price,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,L2-BidSize,L2-AskPrice,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
count,960805.00000,960805.00000,960805.00000,960805.00000,960805.00000,960805.00000,960805.00000,960805.00000,960805.00000,960805.00000,...,960805.00000,960805.00000,960805.00000,960805.00000,960805.00000,960805.00000,960805.00000,960805.00000,960805.00000,960805.00000
mean,0.00074,0.01506,20.41549,20.40796,2399.87394,20.42302,2452.64417,20.39790,3161.48238,20.43315,...,20.50497,2020.14743,20.32762,931.33307,20.53308,2200.69398,20.31753,716.74755,20.57047,2668.61947
std,0.00041,0.00832,1.63248,1.63245,2223.31754,1.63252,2217.52323,1.63250,2073.71869,1.63254,...,1.63414,1931.57670,1.63242,1339.48925,1.63620,2904.33565,1.63241,1505.34699,1.64100,3357.00556
min,0.00043,0.01000,16.35500,16.35000,1.00000,16.36000,1.00000,16.34000,3.00000,16.37000,...,16.43000,1.00000,16.27000,30.00000,16.44000,1.00000,15.24000,2.00000,16.45000,1.00000
25%,0.00048,0.01000,19.01500,19.01000,700.00000,19.02000,700.00000,19.00000,1600.00000,19.03000,...,19.10000,300.00000,18.93000,200.00000,19.12000,200.00000,18.92000,100.00000,19.15000,100.00000
50%,0.00056,0.01000,20.83500,20.83000,1800.00000,20.84000,1800.00000,20.82000,2900.00000,20.85000,...,20.93000,2100.00000,20.75000,400.00000,20.97000,1700.00000,20.74000,200.00000,21.01000,2300.00000
75%,0.00095,0.02000,21.79500,21.79000,3500.00000,21.80000,3600.00000,21.78000,4200.00000,21.81000,...,21.88000,2500.00000,21.71000,900.00000,21.92000,2500.00000,21.70000,500.00000,21.96000,2700.00000
max,0.01985,0.44000,23.46000,23.45000,21611.00000,23.49000,20031.00000,23.44000,25826.00000,23.53000,...,23.93000,16700.00000,23.37000,15700.00000,25.20000,15600.00000,23.36000,15500.00000,26.02000,15600.00000


In [62]:
df_market_l10

,#RIC,Date-Time,rel.spread,abs.spread,mid_price,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,L2-BidPrice,...,L8-AskPrice,L8-AskSize,L9-BidPrice,L9-BidSize,L9-AskPrice,L9-AskSize,L10-BidPrice,L10-BidSize,L10-AskPrice,L10-AskSize
36748,SOXS.K,2025-01-03 09:40:00.006514971-05:00,0.00094,0.02000,21.30000,21.29000,800.00000,21.31000,2600.00000,21.28000,...,21.38000,1900.00000,21.21000,100.00000,21.43000,100.00000,21.20000,100.00000,21.44000,2300.00000
36749,SOXS.K,2025-01-03 09:40:00.016467552-05:00,0.00047,0.01000,21.30500,21.30000,600.00000,21.31000,2600.00000,21.29000,...,21.38000,1900.00000,21.22000,100.00000,21.43000,100.00000,21.21000,100.00000,21.44000,2300.00000
36758,SOXS.K,2025-01-03 09:40:00.076256984-05:00,0.00094,0.02000,21.31000,21.30000,1100.00000,21.32000,1300.00000,21.29000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
36762,SOXS.K,2025-01-03 09:40:00.081235935-05:00,0.00047,0.01000,21.31500,21.31000,600.00000,21.32000,1300.00000,21.30000,...,21.43000,100.00000,21.23000,2000.00000,21.44000,2300.00000,21.22000,100.00000,21.52000,2300.00000
36777,SOXS.K,2025-01-03 09:40:00.116134193-05:00,0.00094,0.02000,21.31000,21.30000,800.00000,21.32000,3900.00000,21.29000,...,21.43000,100.00000,21.22000,100.00000,21.44000,2300.00000,21.21000,100.00000,21.52000,2300.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15803460,SOXS.K,2025-01-31 15:49:58.506118967-05:00,0.00097,0.02000,20.56000,20.55000,3700.00000,20.57000,1300.00000,20.54000,...,20.69000,2400.00000,20.47000,100.00000,20.77000,2400.00000,20.46000,100.00000,21.03000,100.00000
15803466,SOXS.K,2025-01-31 15:49:58.511100868-05:00,0.00049,0.01000,20.56500,20.56000,600.00000,20.57000,600.00000,20.55000,...,20.69000,2400.00000,20.48000,2500.00000,20.77000,2400.00000,20.47000,100.00000,21.03000,100.00000
15803505,SOXS.K,2025-01-31 15:49:58.941228370-05:00,0.00097,0.02000,20.57000,20.56000,1400.00000,20.58000,800.00000,20.55000,...,20.69000,2400.00000,20.48000,2500.00000,20.77000,2400.00000,20.47000,100.00000,21.03000,100.00000
15803518,SOXS.K,2025-01-31 15:49:59.701817156-05:00,0.00049,0.01000,20.56500,20.56000,700.00000,20.57000,2500.00000,20.55000,...,20.69000,2400.00000,20.48000,2500.00000,20.77000,2400.00000,20.47000,100.00000,21.03000,100.00000


In [63]:
df_market_l10[~df_market_l10.isna().any(axis=1)].to_csv(f"data/{ticker}_cleaned_jan2025.csv", index=False)